In [4]:
import pandas  as pd
import numpy   as np
import seaborn as sb
import time    as ti
import itertools

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg         import Vectors
from pyspark.ml.feature        import OneHotEncoderEstimator, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml                import Pipeline

from pyspark.sql               import SparkSession, SQLContext
from pyspark.sql.types         import StructType, StructField, BooleanType, IntegerType, StringType, DoubleType, BinaryType, FloatType
from pyspark.sql.functions     import countDistinct, col, desc

In [2]:
ss = SparkSession.builder\
     .config('spark.executor.memory',       '4G')\
     .config('spark.driver.memory',        '40G')\
     .config('spark.driver.maxResultSize', '10G')\
     .getOrCreate()
sc = ss.sparkContext
sq = SQLContext(sc)

In [5]:
schema = StructType()

for f in range(0, 39 + 1):
    if   f == 0 : schema.add(StructField(f'ctr',        FloatType(), True))
    elif f < 14 : schema.add(StructField(f'i{f   :02}', FloatType(), True))
    else        : schema.add(StructField(f's{f-13:02}',  StringType(), True))

df = sq.read.format('csv')\
            .options(header = 'true', delimiter = '\t')\
            .schema(schema)\
            .load('../data/train.txt')

num_features = [c for c in df.columns if 'i' in c]
cat_features = [c for c in df.columns if 's' in c]

tf           = df.sample(fraction = 0.01, seed = 2019)

In [6]:
trndf,devdf,tstdf = df.randomSplit([0.8,0.1,0.1],seed=42)

In [8]:
trndf.write.parquet('../data/train_w261.parquet')

In [9]:
devdf.write.parquet('../data/dev_w261.parquet')

In [10]:
tstdf.write.parquet('../data/test_w261.parquet')